In [3]:
import argparse
import os
import sys
import numpy as np
import pandas as pd
import h5py
import scanpy as sc
from scipy.sparse import coo_matrix

# parser = argparse.ArgumentParser()
# parser.add_argument('-E', "--expression_path", type=str, required=False, help="Gene expression file.",
#                     default='/sibcb2/bioinformatics2/hongyuyang/dataset/Tres/2.tisch_data/1.neutrophil_data/Gao2024/test.txt')
# parser.add_argument('-D', "--output_file_directory", type=str, required=False, help="Directory for output files.",
#                     default='/sibcb2/bioinformatics2/hongyuyang/dataset/Tres/2.tisch_data/1.neutrophil_data/2.gem_data')
# parser.add_argument('-O', "--output_tag", type=str, required=False, help="Prefix for output files.",
#                     default='Gao2024.Neutrophils')
# args = parser.parse_args()

expression_path = '/sibcb2/bioinformatics2/hongyuyang/dataset/Tres/2.tisch_data/1.neutrophil_data/Gao2024/test.txt'
output_file_directory = '/sibcb2/bioinformatics2/hongyuyang/dataset/Tres/2.tisch_data/1.neutrophil_data/2.gem_data'
output_tag = 'Gao2024.Neutrophils'

if os.path.isdir(expression_path):
    print(f"This dataset don't need process.")
    sys.exit(1)
gem_df = pd.read_csv(expression_path, index_col=0, header=0)


In [66]:
sample_list = []
sample_flag = []
for column in gem_df.columns:
    column_list = column.split('_')
    if 'OA902' in column:
        sample_flag.append(False)
        continue
    sample_flag.append(True)
    if len(column_list) == 2:
        if 'OA089' in column:
            sample_list.append("_".join([column_list[0], column.split('-')[-1]]))
        else:
            sample_list.append(column_list[0])
    elif len(column_list) == 3:
        sample_list.append("_".join(column_list[0:-1]))
    elif len(column_list) == 4:
        sample_list.append("_".join(column_list[0:-1]))
    elif len(column_list) == 5:
        if 'OA091_A012_ICC' in column or 'OA091_A048_ICC' in column:
            sample_list.append("_".join(column_list[0:-1]))
        else:
            sample_list.append("_".join(column_list[0:3]))
    elif len(column_list) == 6:
        sample_list.append("_".join(column_list[0:3]))
    elif len(column_list) == 8:
        sample_list.append("_".join(column_list[0:4]))
    elif len(column_list) == 9:
        sample_list.append("_".join(column_list[0:1] + column_list[-2:]))
    elif len(column_list) == 10:
        sample_list.append("_".join(column_list[0:1]))
gem_filtered = gem_df.loc[:, sample_flag]
gem_filtered.columns = ['.0'.join(['Neutrophils', sample_name, cell_name]) for (sample_name, cell_name) in
                        zip(sample_list, list(gem_filtered.columns)) ]

In [74]:
import numpy as np
import random

# all gene sets
signature = []
fin = open('/sibcb2/bioinformatics2/hongyuyang/dataset/Tres/0.model_file/Tres.kegg')
for l in fin:
    fields = l.strip().split('\t')
    s = fields[2:]
    signature.append(pd.Series(np.ones(len(s)), index=s, name=fields[0]))
fin.close()

# 此时signature是每个kegg通路的基因名，行为通路，列为基因
signature = pd.concat(signature, axis=1, join='outer', sort=False) # 将通路的所有基因都作为一列，形成一个基因数*通路数的矩阵
signature.fillna(0, inplace=True) # 将通路中不存在的基因的值由nan变成0
all_gene_list = list(signature.index)

random_geneset = []
for i in range(100):
    random_gene = random.choice(all_gene_list)
    if random_gene not in random_geneset:
        random_geneset.append(random_gene)

with open('/sibcb2/bioinformatics2/hongyuyang/dataset/Tres/0.model_file/Tres_kegg.Neutrophils_random.txt', mode='a') as file:
    file.write('Neutrophils_signature\thttps://www.cell.com/cancer-cell/fulltext/S1535-6108(23)00433-6')
    for gene in random_geneset:
        file.write('\t' + gene)

In [1]:
import pandas as pd

df = pd.DataFrame({
    'A': [1, 2, 3, 4, 5],
    'B': [5, 4, 3, 2, 1],
    'C': [2, 3, 4, 5, 6]
})
df

,A,B,C
0,1,5,2
1,2,4,3
2,3,3,4
3,4,2,5
4,5,1,6


In [3]:
df = df.subtract(df.mean(axis=1), axis=0)
df

,A,B,C
0,-1.666667,2.333333,-0.666667
1,-1.000000,1.000000,0.000000
2,-0.333333,-0.333333,0.666667
3,0.333333,-1.666667,1.333333
4,1.000000,-3.000000,2.000000
